In [4]:
import pandas as pd
import ast
from collections import Counter
import re
from sklearn.model_selection import train_test_split
from KBEB_extraction import ID_linkages
import json
from tqdm import tqdm

## Creation of KBs and variations (IDs)

In [7]:
#name_list = ["wine chicken"]  #Mulled Wine
#df_samecountry = df_train[df_train['countries'] == 'france']
#list_title = list(df_samecountry['title'])
#temp_ids = []
#for j, title in enumerate(list_title):
#    title = title.lower()
#    for name in name_list:
#        name = name.lower()
#        if name in title:
#            temp_ids.append(j)
#
#list_of_ids = list(df_samecountry.iloc[temp_ids]["Unnamed: 0"])
#print(list_of_ids)

#print(list(df_train[df_train['Unnamed: 0'] == 2108435]['title']))

#df_train[df_train['Unnamed: 0'] == 2108435]

Ici on consitue la KB/EB à partir du fichier de train que l'on a généré

Le fichier de validation sera pour constituer des variations qui nous permettront d'optimiser les hyperparamètres des modèles (Écart moyen des points et de faire une petite validation des rankings)

Le fichier de tests aura les variations qu'on fera labellées -- on doit s'assurer ici qu'on a assez de variations en tests

In [10]:
df_names = pd.read_excel("Recipe_dataset/FC0924_Selected_recipes_VNewCoutrniesCorrected.xlsx")
df_train = pd.read_csv("Recipe_dataset/NLGRecipe_KownCountriesOnly_Train.csv")
df_valid = pd.read_csv("Recipe_dataset/NLGRecipe_KownCountriesOnly_Valid.csv")
df_test = pd.read_csv("Recipe_dataset/NLGRecipe_KownCountriesOnly_Test.csv")

In [12]:
# The one we keep -- As decided with Romain the KB and the EB -- with all recipe of the same country with same name
collect_KB = ID_linkages(df_names)
df_names_long = collect_KB.ID_list_longKB(df_train)

In [14]:
df_names_long_trainvar = collect_KB.train_variations(df_names_long, df_train)
df_names_long_validvar = collect_KB.test_variations(df_names_long_trainvar, df_valid, col_name='valid_variations')
df_names_long_testvar = collect_KB.test_variations(df_names_long_validvar, df_test, col_name='test_variations')
df_names_long_testvar.head()

,Recette,Pays,ids_LongList,train_variations,valid_variations,test_variations
0,Honey Cake,ukraine,"[2060432, 2203354, 1077095, 1143226, 1902200, ...","[1609071, 1302873, 1182851, 1131316, 1095143, ...","[517917, 2080408, 1041694, 218055]","[1936434, 1008416, 331579, 545002, 1407223, 12..."
1,Sesame Noodle,thailand,"[372599, 2159359, 1873151, 298695, 1122236, 18...","[1227441, 1845726, 1045952, 1114312, 1609376, ...","[1393403, 1280133]","[989404, 2085811, 1387994, 1621723, 1073752]"
2,Marinade,south korea,"[1285303, 1993067, 1744664, 1114136, 1700445, ...","[1185521, 1111148, 1947504, 1542327, 980168, 1...","[2078789, 1608872, 1314019, 414394, 1356146, 1...","[1064233, 1301636, 924326, 1043037, 607106, 10..."
3,Chicken Satay | Satay Chicken,thailand,"[1359368, 925716, 1596440, 2048011, 1343719, 9...","[1054353, 1036325, 210380, 1180698, 1319376, 9...","[1840402, 1637506]","[984943, 1224116, 2021347, 1042459, 1063697, 1..."
4,Tartar,japan,"[916180, 1231158, 1870929, 1026618]","[1592080, 963214, 1281797, 1153019, 1226450, 1...","[1986345, 2061773, 1231640, 957926, 1325377, 1...","[1991897, 1393889]"


In [16]:
#col_list = ['']
#df = df.drop(col_list, axis=1)

In [22]:
df_names_long_testvar.to_csv('Recipe_dataset/NewRecipesKBAndVar.csv',index=False)

## Getting recipes texts, list of ingredients, clean texts and saving everything in 1 Json per RecipesNames

In [20]:
import pandas as pd
import ast
from collections import Counter
import re
from sklearn.model_selection import train_test_split
from KBEB_extraction import ID_linkages
import json
from tqdm import tqdm
from json_dataset import dataset_generation

In [24]:
df_names_new  = pd.read_csv("Recipe_dataset/NewRecipesKBAndVar.csv")
df_train = pd.read_csv("Recipe_dataset/NLGRecipe_KownCountriesOnly_Train.csv")
df_valid = pd.read_csv("Recipe_dataset/NLGRecipe_KownCountriesOnly_Valid.csv")
df_test = pd.read_csv("Recipe_dataset/NLGRecipe_KownCountriesOnly_Test.csv")

In [28]:
gen_instance = dataset_generation(df_names_new, df_train, df_valid, df_test, lemma="True", authrorized_pos= ['PROPN', 'PRON', 'ADJ', 'ADV', 'NOUN', 'NUM', 'VERB'])

In [30]:
gen_instance.json_create(file_path='./Recipe_dataset/NewRecipes/')

===========Starting Data Collection============
=====IDs collected=====


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 41.86it/s]


=====Recipes, countries and ingredients collected=====


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.66s/it]


=====cleaning recipes done=====
============Starting saving procedure============


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 323.30it/s]

============ALL 5 FILES SAVED============
